<a href="https://colab.research.google.com/github/haydenclose/Cloud_based_Oil_Detection/blob/main/Cloud_Based_Analysis_of_Oil_Spills_from_Wrecks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cloud Based Analysis of Oil Spills from Wrecks

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Set-up of workspace
Import Earth Engine API (`import ee`).
Additionally, import useful python packages from examplar scripts (need to remove redundant ones after)

In [4]:
import ee
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd                                                             # Useful package to read in csv's etc...
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell. Authorisation last a week.

In [5]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Eg2uvHZH9-DIhbJvF5uo5519GzDMXCAxZid_zWr8mIk&tc=kFKVg7xQ6NemrEwhLJfbxrCGkebahuUXCITIZT-FKCM&cc=J05hAqfFdgem4ZGT11X199-wVFBxuk1wy1b3UhqaWWw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk4jw-xF_EdcfkMcgkOyc-higSza-kjsxm7dAK3VNVnSyiW4pIJ6zdc

Successfully saved authorization token.


## Interactive base map

The [`folium`](https://python-visualization.github.io/folium/)
library can be used to display `ee.Image` objects on an interactive
[Leaflet](https://leafletjs.com/) map. Folium has no default
method for handling tiles from Earth Engine, so one must be defined
and added to the `folium.Map` module before use.

The following cell provides an example of adding a method for handing Earth Engine
tiles and using it to display an elevation model to a Leaflet map

In [16]:
# Import the Folium library.
import folium                                                                   # Used for Interactive mapping

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,                                                                # Adds the layer name to the radio buttons rather than web address
    overlay = True,                                                             # Allow layers to overlay upon each
    control = True                                                              # Allows the user to turn on and off layers
  ).add_to(self)                                                                # Allows layers to be added after this base layer

# Add EE drawing method to folium, to be able to add other layers to this base map (see examplar below)
folium.Map.add_ee_layer = add_ee_layer                                         

from folium.plugins import Draw                                                 # Toolbox for adding the toolbar
draw = Draw(export=True)                                                        # Adds toolbars to folium plots

print('This sets up te base parameters of the map with no output')


This sets up te base parameters of the map with no output


Now lets make an examplar interactive map just with the base layer

In [86]:

lat, lon = 52, 0                                                                # Define the Lat and Lon to centre the map on (UK)
my_map = folium.Map(location=[lat, lon], zoom_start=7)                          # Create the map zoomed to level 7 on our defined Lat and Lon (not yet ouputted)

draw.add_to(my_map)                                                             # Add the draw toolbar to the map

# Display the map
display(my_map)

### Annotating map with points
We have a list of wreck locations (stored on CDP). I have uploaded the file called 'Wreck Database_V2.2.xls' to my google drive. 

Now we use pandas to read the excel file and the `Wreck.head()` to view the first few rows of the dataframe. Note the ***magic wand*** button that allows for the interaction with the data



In [124]:
Wrecks = pd.read_excel('Wreck Database_V2.2.xls')

Wrecks.head()

,Owner,Wreck_ID,Catergory,Geographical area,Lattitude,Longitude,DECLat,DECLong,Depth,Dates of service,History of leaks,Cargo,Physical Environment,Background info on wreck,Sources
0,MOD,SS DERBENT,green,Wales (UK),53° 28’ 24.206’’N,004° 14’ 08.390’’W,53.473390,-4.235660,45m,NaN,Not known,3700 tons of fuel oil,"Seabed type is not known, however, EUSeamap, s...",\nSS Derbent was a 3178 ton tanker 95 m in len...,See full wreck assessment MOD CDP
1,MOD,HMS PRINCE OF WALES,green,Malaysia,03°34’3.30”N,104°27’51.78”E,3.517583,104.464383,~68m,NaN,Neither wreck location showed any indication o...,NaN,NaN,NaN,20140328-MYEEBX09CA_4376-U.pdf
2,MOD,HMS REPULSE,green,Malaysia,03°37’14.23”N;,104°20’42.65”E,3.620619,104.345181,~68m,NaN,Neither wreck location showed any indication o...,NaN,NaN,NaN,20140328-MYEEBX09CA_4376-U.pdf
3,MOD,RFA WAR MEHTAR,green,Southern North Sea (UK),52° 36.3'N,002° 8.9'E,52.605000,2.148330,Between 26.5 m and 40 m with 2-3 m scour.,NaN,Yes chronic leaks reported since 2014 through...,7000 tonnes Admiralty fuel oil,The wreck lies in an area of high tidal action...,RFA War Mehtar was a 5520 ton tanker 122 m in ...,See full wreck assessment MOD CDP
4,MOD,RFA ATHELSTANE,green,Sri Lanka,7 19.951 N,81 56.376E,7.332520,81.939600,42m,NaN,Not known,6096 tonnes Admiralty fuel oil,Seafloor sediment is clay (Dutkiewicz et al. 2...,"This assessment considers RFA Athelstane, a RF...",NaN


### Adding the wrecks to the map
Here we use a little `for` loop to go through each row of the wreck dataframe and catergorise by priority.

This method also adds the wrecks to the `Folium.FeatureGroup`  so we can turn on and off the wrecks we do and dont want in the feature control panel.

Finally we can add a `popup`, this can be an extensive table but for now to keep simple it is just the wrecks name.

In [131]:
Point_map = folium.Map(location=[lat, lon], zoom_start=7)                       # Base map with zoomed location and level.
for Wreck_group, Wreck in Wrecks.groupby('Catergory'):                          # For loop to group the wrecks together by priority
    feature_group = folium.FeatureGroup(Wreck_group)                            # Add the wreck groups to the feature control panel
    for row in Wreck.itertuples():                                              # Loop through the individual wrecks to plot
        folium.CircleMarker(location=[row.DECLat, row.DECLong],                 # Adds circle marker by lat and long
                            popup=row.Wreck_ID,                                 # If click a point, tells us the name of the wreck
                            radius = 3,                                         # Size of point
                            fill_color = row.Catergory,                         # Colour by catergory
                            fill_opacity=1,                                     # Opacity of the point
                            color=row.Catergory).add_to(feature_group)          # Colour of outline by catergory
    feature_group.add_to(Point_map)                                             # Adds the points to the map 

folium.LayerControl().add_to(Point_map)                                         # Add the layer control to the map
Point_map                                                                       # Plots the map

## Sentinel-1 imagery



Below is the area of HMS Repulse and HMS Prince of Wales
Just copied and pasted from the `geoJSON` website

### User guide on sentinel imagery
good [`userguide`](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-1) here




### Selecting an area of interest
The images are massive so want a spatial subset. A convenient way the area of interest (AOI) is to use the geojson.io website, from which we can cut and paste the corresponding [`GeoJSON`](https://geojson.io/#map=2/20/0) object description.

Below is the area of HMS Repulse and HMS Prince of Wales of Malaysia

In [221]:
geoJSON  = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              103.50750822030545,
              4.231086748021667
            ],
            [
              103.50750822030545,
              3.076337170071014
            ],
            [
              105.14317628353797,
              3.076337170071014
            ],
            [
              105.14317628353797,
              4.231086748021667
            ],
            [
              103.50750822030545,
              4.231086748021667
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

Next, we filter the S1 archive to get an image over the aoi acquired sometime in June, 2015. Any old image will do fine, so we won't bother to specify the orbit number or whether we want the ASCENDING or DESCENDING node. If we don't specify the instrument mode or resolution, we get IW (interferometric wide swath) mode and 10 X 10 m pixels by default. 

Below I have selected the Sentinel-1 ground range detected images, if want the images converted to decibels can use `ee.ImageCollection('COPERNICUS/S1_GRD')`

# upto here figuring out how to get an image

In [229]:
im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD')                        # Ground range detected images   
           .filterBounds(aoi)                                                   # Filter by Area of Interest (AOI)
           .filterDate(ee.Date('2015-07-20'),ee.Date('2016-07-21'))             # Filter by dates
           .sort('date')
           .first()                                                       # Sort by date
           .clip(aoi))

# Extract timestamps and print
#timestamplist = (im_coll.aggregate_array('date')                                
#                 .map(lambda d: ee.String('T').cat(ee.String(d)))
#                 .getInfo())
#timestamplist

In [211]:
im_coll = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')) 
                       .first() 
                       .clip(aoi))

It will turn out to be convenient to work with a list rather than a collection, so we'll convert the collection to a list and, while we're at it, clip the images to our AOI:

In [223]:
def clip_img(img):
    """Clips a list of images."""
    return ee.Image(img).clip(aoi)

im_list = im_coll.toList(im_coll.size())
im_list = ee.List(im_list.map(clip_img))
print('No of images in date range =',im_list.length().getInfo())


AttributeError: ignored

In [ ]:
url = im_coll.select('VV').getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=800)

In [66]:
def selectvv(current):
    return ee.Image(current).select('VV')

vv_list = im_list.map(selectvv)

location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(location=location, zoom_start=13)
rgb_images = (ee.Image.rgb(vv_list.get(1), vv_list.get(2), vv_list.get(3))
              .log10().multiply(10))
mp.add_ee_layer(rgb_images, {'min': -20,'max': 0}, 'rgb composite')
draw.add_to(mp)
mp.add_child(folium.LayerControl())